In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps
import PIL
import torch, torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from ansim_dataset import ansimDataset, create_circular_mask
from convolution_lstm import encoderConvLSTM, decoderConvLSTM
import random
import math
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import time
import os

In [2]:
img_path = '/home/rliu/ansim/data/data/JPEGImages/'
img_list_csv = '/home/rliu/github/ansim/img_list.csv'
train_csv = '/home/rliu/github/ansim/train.csv'
test_csv = '/home/rliu/github/ansim/test.csv'

In [3]:
mask = create_circular_mask(128,128)
trainset = ansimDataset(img_list_csv = img_list_csv, seq_csv = train_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
trainloader = torch.utils.data.DataLoader(trainset,
                                             batch_size=8, shuffle=True,
                                             num_workers=4)

testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
testloader = torch.utils.data.DataLoader(testset,
                                             batch_size=8, shuffle=True,
                                             num_workers=4)

In [4]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("GPU in use")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

GPU in use


In [5]:
def train_model(encoder, decoder, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_encoder_wts = encoder.state_dict()
    best_decoder_wts = decoder.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training phase
        scheduler.step()
        encoder.train(True)  # Set model to training mode
        decoder.train(True)  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        trainset = ansimDataset(img_list_csv = img_list_csv, seq_csv = train_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
        trainloader = torch.utils.data.DataLoader(trainset,
                                                     batch_size=1, shuffle=True,
                                                     num_workers=4)

        print("trainloader ready!")
        testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
        testloader = torch.utils.data.DataLoader(testset,
                                                     batch_size=1, shuffle=True,
                                                     num_workers=4)
        print("testloader ready!")
        
        for data in trainloader:
            # get the inputs
            data_split = torch.split(data, 10, dim=1)
            inputs = data_split[0]
            target = data_split[1]
#            print(inputs)
            # wrap them in Variable
            if use_gpu:
#                inputs = Variable(inputs.cuda())
#                labels = Variable(labels.cuda())
#                inputs = torch.nn.DataParallel(inputs, device_ids=[0, 1]).cuda()
#                labels = torch.nn.DataParallel(labels, device_ids=[0, 1]).cuda()
                inputs, target = inputs.to(device), target.to(device)
#                print(inputs)
            else:
                inputs, target = Variable(inputs), Variable(target)

            # zero the parameter gradients
            optimizer.zero_grad()
            
            output, h, c, states = encoder(inputs)
            output_last = output[0][0].double()
            h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
            c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
            states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
            x = decoder.activateConv(states_cat)
            input_d = [x, states]
            output_d, h_d, c_d, states_d = decoder(input_d)
            predicted = torch.cat(output_d, dim=0, out=None).double()
            
            
            loss = criterion(predicted, target)
            # forward
#            outputs = model(inputs)
#            _, preds = torch.max(outputs.data, 1)
#            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            # statistics
            iter_loss = loss.item()
            running_loss += loss.item()    
            epoch_loss = running_loss / len(trainset)
            
            print('{} Loss: {:.4f} batch_loss: {:d}'.format(
                "train", epoch_loss, iter_loss))
        
        with torch.no_grad():
            for data in testloader:
                data_split = torch.split(data, 10, dim=1)
                inputs = data_split[0]
                target = data_split[1]
                
                if use_gpu:
                    inputs, target = inputs.to(device), target.to(device)
                else:
                    inputs, target = Variable(inputs), Variable(target)

                    
                output, h, c, states = encoder(inputs)
                output_last = output[0][0].double()
                h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
                c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
                states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
                x = decoder.activateConv(states_cat)
                input_d = [x, states]
                output_d, h_d, c_d, states_d = decoder(input_d)
                predicted = torch.cat(output_d, dim=0, out=None).double()
            
                loss_test = criterion(predicted, target)
                iter_loss_test = loss_test.item()
                running_loss_test += loss_test.item()    
                epoch_loss_test = running_loss_test / len(testset)

        print('Loss on the test images: %.5f %%' % (
            epoch_loss_test))
        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    encoder.load_state_dict(best_encoder_wts)
    decoder.load_state_dict(best_decoder_wts)
    return encoder, decoder

In [6]:
# transfer learning resnet18
step_in = 10
step_out= 10
encoder = encoderConvLSTM(input_channels=1, hidden_channels=[128, 64, 64, 32, 32], kernel_size=3, step=step_in, effective_step=range(10))
decoder = decoderConvLSTM(input_channels=1, hidden_channels=[128, 64, 64, 32, 32], kernel_size=3, step=step_out, effective_step=range(10))

if use_gpu:
#     encoder = torch.nn.DataParallel(encoder)
#     decoder = torch.nn.DataParallel(decoder)
    encoder.to(device)
    decoder.to(device)

criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(decoder.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

# train model
encoder,decoder = train_model(encoder,decoder, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)
# torch.save(encoder, output_path)

Epoch 0/9
----------
trainloader ready!
testloader ready!


ValueError: too many values to unpack (expected 4)

In [16]:
input = Variable(torch.randn(1,10, 1, 128, 128)).cuda()

In [18]:
input.shape

torch.Size([1, 10, 1, 128, 128])